# Evaluate Partial Charge Prediction
Test how well our models for partial charges are working

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
from jcesr_ml.mpnn import set_custom_objects, run_model
from jcesr_ml.benchmark import load_benchmark_data
from sklearn.metrics import mean_absolute_error
from keras.models import load_model
from ase.units import eV, Hartree
from time import perf_counter
from rdkit.Chem import AllChem
from rdkit import Chem
from pathlib import Path
from tqdm import tqdm
from glob import glob
import pickle as pkl
import pandas as pd
import numpy as np
import torch
import json
import os

Using TensorFlow backend.


In [2]:
set_custom_objects()

## Load in Datasets
We need the datasets for determining the shape of the inputs

In [3]:
test_data = pd.read_pickle('mapped_charges_dataset.pkl.gz')

In [4]:
test_data.query('in_holdout', inplace=True)

## Get Baselines
See how good assuming all networks to be uncharged is.

In [5]:
charge_mae = test_data['atomic_charges'].apply(np.array).apply(np.abs).apply(sum).sum() / test_data['n_atom'].sum()
print(f'MAE of all charges {charge_mae: .2f}')

MAE of all charges  0.19


Estimate partial charges with [Gasteiger Charges](https://www.rdkit.org/docs/source/rdkit.Chem.rdPartialCharges.html)

In [6]:
def compute_gasteiger_charges(smiles):
    """Compute the Gasteiger partial charges for a molecule
    
    Args:
        smiles (str): SMILES string of the molecule
    Returns:
        (ndarray) Charges on each atom
    """
    
    # Parse the SMILES string
    mol = Chem.MolFromSmiles(smiles)
    mol = Chem.AddHs(mol)
    
    # Compute the charges
    AllChem.ComputeGasteigerCharges(mol)
    
    # Extract the charges
    return np.array([float(atom.GetProp('_GasteigerCharge')) for atom in mol.GetAtoms()])

In [7]:
%%time
test_data['gasteiger_charges'] = test_data['smiles_0'].apply(compute_gasteiger_charges)

CPU times: user 6.03 s, sys: 5.48 ms, total: 6.03 s
Wall time: 6.03 s


In [8]:
gastier_charges_mae = mean_absolute_error(np.hstack(test_data['gasteiger_charges']), np.hstack(test_data['mapped_charges']))
print(f'Gastier MAE: {gastier_charges_mae:.3f}')

Gastier MAE: 0.124


## Evaluate All Models
See if our deep learning model is better than the Gasteiger partial charges

In [9]:
models = glob(os.path.join('networks', '**', 'best_model.h5'), recursive=True)

In [10]:
def score_model(path):
    """Given a log file, parse the settings for the network and the epoch time / ending val_loss
    
    Args:
        path (str): Get the path 
    """
    
    # Get the metadata from directory structure
    path = Path(path)
    parents = list(path.parents)
    metadata = [p.name for p in parents[:-3]]
    metadata = dict([(x[-1], '-'.join(x[:-1])) for x in map(lambda x: x.split("-"), metadata)])
    metadata['path'] = str(path.parent)
    metadata['network'] = parents[-3].name
    
    # Convert numerical values
    for k in ['nodes', 'entries', 'batch_size']:
        metadata[k] = int(metadata[k])
    
    # Score the model on the target variable
    with open(parents[-3].joinpath('options.json')) as fp:
        options = json.load(fp)
    output = options['output_props']
    metadata['output'] = output
    
    # Load in the log
    log = pd.read_csv(path.parent.joinpath('log.csv'))
    metadata['epochs'] = len(log)
    metadata['median_epoch_time'] = np.percentile(log['epoch_time'], 50)
    metadata['total_time'] = log['epoch_time'].sum()
    metadata['cpu-hrs'] = metadata['total_time'] * metadata['nodes'] / 3600 * 64
    metadata['best_loss'] = log['val_loss'].min()
    metadata['best_loss_epoch'] = log['val_loss'].idxmin()
    
    # Check whether the network had finished training
    metadata['finished'] = os.path.isfile(path.parent.joinpath('finished'))
    
    # Load the converter
    with open(parents[-3].joinpath('converter.pkl'), 'rb') as fp:
        converter = pkl.load(fp)
    
    # Load in the model and run it on the test set
    directory = path.parent
    y_true = np.hstack(test_data['mapped_charges'])
    for name in ['best_model.h5', 'checkpoint.h5']:
        tag = '-best' if name.startswith('best') else '-last'
        
        model = load_model(str(path.parent.joinpath(name)))
        start_time = perf_counter()
        y_pred = np.squeeze(run_model(model, converter, test_data['smiles_0'], chunk_size=1024))
        metadata[f'mae{tag}'] = mean_absolute_error(y_pred, y_true)
        metadata[f'eval_time{tag}'] = perf_counter() - start_time
        
    return metadata

In [13]:
results = pd.DataFrame([score_model(m) for m in tqdm(models) if 'standard' in m])


  0%|          | 0/4 [00:00<?, ?it/s]/home/lward/miniconda3/envs/jcesr_ml/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "

 25%|██▌       | 1/4 [00:32<01:36, 32.23s/it]/home/lward/miniconda3/envs/jcesr_ml/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/lward/miniconda3/envs/jcesr_ml/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shap

In [14]:
results[['network', 'learning_rate', 'batch_size', 'mae-best', 'mae-last', 'cpu-hrs', 'total_time']]

,network,learning_rate,batch_size,mae-best,mae-last,cpu-hrs,total_time
0,standard,1.00e-04,1024,0.016461,0.017548,110.938707,6240.302262
1,standard,1.00e-04,8192,0.009501,0.009756,2113.665788,14861.712574


*Finding*: Our "standard" network does pretty well. MAEs less than 10% of the variation in the dataset and better than the Gasteiger charges.